In [76]:
import os
import pandas as pd
os.chdir('/Users/rishi/Desktop/DAEN 690/glassdoor')
import string

In [77]:
#to process the text
import nltk
from nltk.tokenize import sent_tokenize
#pos tagging
from nltk import pos_tag
import string
from nltk.corpus import stopwords
#to create frequency distribution
from nltk.probability import FreqDist
#to create lemmas of the words
from nltk.stem import WordNetLemmatizer
#to perform operations using regular expressions
import re

In [78]:
#to generate ngrams of text
from nltk.util import ngrams

In [79]:
total=pd.read_csv('total_jobs.csv')

In [80]:
whole_df = total[['title','summary']]

In [81]:
#Text Preprocessing
#Sentence tokenization
def tokenize_sentences(textContent):
    #No of sentences
    sentences = nltk.sent_tokenize(textContent.lower())
    return sentences

#Word Tokenization
def tokenize_words(textContent):
    tokenized_words = nltk.word_tokenize(textContent)
    return tokenized_words

#remove punctuations
def remove_punctuation(textContent):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    punc_removed_words = tokenizer.tokenize(textContent.lower())
    return punc_removed_words

#remove stop words
#In this scenario, pass words obtained after removing punctuation
def remove_stopwords(word_list):
    stopwords_removed = []  
    stop_words = set(stopwords.words('english'))
    for w in word_list:  
        if w not in stop_words:  
            stopwords_removed.append(w) 
    return stopwords_removed

#Remove digits
#In this scenario, pass word list obtained after removing stop words
#def remove_digits_from_words(word_list):
#    digits_removed = []
#    for w in word_list:
#        word = re.sub(r'[0-9]+', '', w)
#        digits_removed.append(word)
#   return digits_removed

#lemmatization
#in this scenario, pass words list obtained after removing stopwords
def create_lemmas(word_list):
    lemmatizer = WordNetLemmatizer() 
    lemmatized_words = []
   
    for word, tag in pos_tag(word_list):
        wntag = tag[0].lower()
        #POS tags 
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        lemma = lemmatizer.lemmatize(word, wntag) if wntag else word
        lemmatized_words.append(lemma)
    return lemmatized_words

In [82]:
#Perform Text processing
def get_preprocessed_content(row, text_from, text_preprocess_category):
    
    textContent = row[text_from]
    if text_preprocess_category == 'sentences':
        sentence_tokens = tokenize_sentences(textContent)
        return sentence_tokens
    elif text_preprocess_category == 'word_tokens':
        w_tokens = tokenize_words(textContent)
        return w_tokens
    elif text_preprocess_category == 'words_no_punctuation':
        no_punc_words = remove_punctuation(textContent)
        return no_punc_words
    else:
        return None

In [83]:
#get tokens/ stopwords/ lemmas
def get_preprocessed_tokens(row, tokens_from, text_preprocess_category):
    
    token_list = row[tokens_from]
    
    if text_preprocess_category == 'words_no_sw':
        no_stop_words = remove_stopwords(token_list)
        return no_stop_words
    elif text_preprocess_category == 'words_no_digits':
        no_digits = remove_digits_from_words(token_list)
        return no_digits
    elif text_preprocess_category == 'lemmas':
        lemmas = create_lemmas(token_list)
        return lemmas
    else:
        return None

In [84]:
preprocessing_actions_one = ['sentences','word_tokens','words_no_punctuation']
preprocessing_actions_two = ['words_no_sw','lemmas']

In [85]:
# for total data
whole_title_df = pd.DataFrame()

for pa in preprocessing_actions_one:
    whole_title_df[pa] = whole_df.apply(lambda row: get_preprocessed_content(row, 'summary',pa), axis=1)
    
for pa in preprocessing_actions_two:
    if pa == 'words_no_sw':
        tokens_from = 'words_no_punctuation'
    elif pa == 'lemmas':
        tokens_from = 'words_no_sw'
    whole_title_df[pa] = whole_title_df.apply(lambda row: get_preprocessed_tokens(row,tokens_from ,pa), axis=1)

In [86]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))
        
        
# Driver code    
whole_title_df['full']= whole_title_df.apply(lambda row: listToString(row['lemmas']), axis=1)


In [87]:
whole_title_df

,sentences,word_tokens,words_no_punctuation,words_no_sw,lemmas,full
0,"[college degree (associates or bachelors)., mu...","[College, degree, (, Associates, or, Bachelors...","[college, degree, associates, or, bachelors, m...","[college, degree, associates, bachelors, must,...","[college, degree, associate, bachelor, must, a...",college degree associate bachelor must authori...
1,[assist with the management and coordination b...,"[Assist, with, the, management, and, coordinat...","[assist, with, the, management, and, coordinat...","[assist, management, coordination, tracking, s...","[assist, management, coordination, track, syst...",assist management coordination track system ve...
2,[train other team members on gis and other gis...,"[Train, other, team, members, on, GIS, and, ot...","[train, other, team, members, on, gis, and, ot...","[train, team, members, gis, gis, technologies,...","[train, team, member, gi, gis, technology, pro...",train team member gi gis technology provide gu...
3,[bs/ ms in it or equivalent is required with s...,"[BS/, MS, in, IT, or, equivalent, is, required...","[bs, ms, in, it, or, equivalent, is, required,...","[bs, ms, equivalent, required, strong, academi...","[b, ms, equivalent, require, strong, academic,...",b ms equivalent require strong academic backgr...
4,[experience with using a git workflow for soft...,"[Experience, with, using, a, Git, workflow, fo...","[experience, with, using, a, git, workflow, fo...","[experience, using, git, workflow, software, d...","[experience, use, git, workflow, software, dev...",experience use git workflow software developme...
...,...,...,...,...,...,...
45088,[position description what does a great softw...,"[Position, Description, What, does, a, great, ...","[position, description, what, does, a, great, ...","[position, description, great, software, devel...","[position, description, great, software, devel...",position description great software developmen...
45089,[job title : software developer / analyst jo...,"[Job, Title, :, Software, Developer, /, Analys...","[job, title, software, developer, analyst, job...","[job, title, software, developer, analyst, job...","[job, title, software, developer, analyst, job...",job title software developer analyst job respo...
45090,[swbc has been recognized as one of the best c...,"[SWBC, has, been, recognized, as, one, of, the...","[swbc, has, been, recognized, as, one, of, the...","[swbc, recognized, one, best, companies, work,...","[swbc, recognize, one, best, company, work, te...",swbc recognize one best company work texas exc...
45091,[summary\n\ncreates and maintains manual and a...,"[SUMMARY, Creates, and, maintains, manual, and...","[summary, creates, and, maintains, manual, and...","[summary, creates, maintains, manual, automate...","[summary, creates, maintain, manual, automated...",summary creates maintain manual automated test...


In [88]:
#Ngram
def generate_ngrams(text,n):
    n_grams = ngrams(nltk.word_tokenize(text.lower()), n)
    return [' '.join(grams) for grams in n_grams]

whole_title_df['tri-grams'] = whole_title_df.apply(lambda row: generate_ngrams(row['full'],3), axis=1)
whole_title_df['bi-grams'] = whole_title_df.apply(lambda row: generate_ngrams(row['full'],2), axis=1)

In [89]:
whole_title_df

,sentences,word_tokens,words_no_punctuation,words_no_sw,lemmas,full,tri-grams,bi-grams
0,"[college degree (associates or bachelors)., mu...","[College, degree, (, Associates, or, Bachelors...","[college, degree, associates, or, bachelors, m...","[college, degree, associates, bachelors, must,...","[college, degree, associate, bachelor, must, a...",college degree associate bachelor must authori...,"[college degree associate, degree associate ba...","[college degree, degree associate, associate b..."
1,[assist with the management and coordination b...,"[Assist, with, the, management, and, coordinat...","[assist, with, the, management, and, coordinat...","[assist, management, coordination, tracking, s...","[assist, management, coordination, track, syst...",assist management coordination track system ve...,"[assist management coordination, management co...","[assist management, management coordination, c..."
2,[train other team members on gis and other gis...,"[Train, other, team, members, on, GIS, and, ot...","[train, other, team, members, on, gis, and, ot...","[train, team, members, gis, gis, technologies,...","[train, team, member, gi, gis, technology, pro...",train team member gi gis technology provide gu...,"[train team member, team member gi, member gi ...","[train team, team member, member gi, gi gis, g..."
3,[bs/ ms in it or equivalent is required with s...,"[BS/, MS, in, IT, or, equivalent, is, required...","[bs, ms, in, it, or, equivalent, is, required,...","[bs, ms, equivalent, required, strong, academi...","[b, ms, equivalent, require, strong, academic,...",b ms equivalent require strong academic backgr...,"[b ms equivalent, ms equivalent require, equiv...","[b ms, ms equivalent, equivalent require, requ..."
4,[experience with using a git workflow for soft...,"[Experience, with, using, a, Git, workflow, fo...","[experience, with, using, a, git, workflow, fo...","[experience, using, git, workflow, software, d...","[experience, use, git, workflow, software, dev...",experience use git workflow software developme...,"[experience use git, use git workflow, git wor...","[experience use, use git, git workflow, workfl..."
...,...,...,...,...,...,...,...,...
45088,[position description what does a great softw...,"[Position, Description, What, does, a, great, ...","[position, description, what, does, a, great, ...","[position, description, great, software, devel...","[position, description, great, software, devel...",position description great software developmen...,"[position description great, description great...","[position description, description great, grea..."
45089,[job title : software developer / analyst jo...,"[Job, Title, :, Software, Developer, /, Analys...","[job, title, software, developer, analyst, job...","[job, title, software, developer, analyst, job...","[job, title, software, developer, analyst, job...",job title software developer analyst job respo...,"[job title software, title software developer,...","[job title, title software, software developer..."
45090,[swbc has been recognized as one of the best c...,"[SWBC, has, been, recognized, as, one, of, the...","[swbc, has, been, recognized, as, one, of, the...","[swbc, recognized, one, best, companies, work,...","[swbc, recognize, one, best, company, work, te...",swbc recognize one best company work texas exc...,"[swbc recognize one, recognize one best, one b...","[swbc recognize, recognize one, one best, best..."
45091,[summary\n\ncreates and maintains manual and a...,"[SUMMARY, Creates, and, maintains, manual, and...","[summary, creates, and, maintains, manual, and...","[summary, creates, maintains, manual, automate...","[summary, creates, maintain, manual, automated...",summary creates maintain manual automated test...,"[summary creates maintain, creates maintain ma...","[summary creates, creates maintain, maintain m..."
